爬取新乡市的历史天气数据，并且用在模型中，实例代码：https://yonniye.com/archives/11.html     2345里面新乡的网页：https://tianqi.2345.com/wea_history/53986.htm

In [1]:
import requests
import urllib.request
from tqdm import tqdm
from keys import *
import os

In [2]:
months = []
year = 2020
for i in range(12):
    months.append("%d%02d"%(year, i+1))
year = 2021
for i in range(12):
    months.append("%d%02d"%(year, i+1))
year = 2022
for i in range(10):
    months.append("%d%02d"%(year, i+1))
selected_month = months[22]


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
     Chrome/77.0.3865.120 Safari/537.36"
}


todo_url = f"http://tianqi.2345.com/t/wea_history/js/{selected_month}/58238_{selected_month}.js"

In [10]:
def get_history_temps(months):
    """
        这个函数可以按照月份需求读取2345天气网上的历史数据，并搜索得到历史上某一天的最高温和最低温
        months输入为列表，格式为%yyyy%mm
        原始方法：https://yonniye.com/archives/11.html
    """
    date_list = []
    temp_high_list = []
    temp_low_list = []
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
     Chrome/77.0.3865.120 Safari/537.36"}
    for selected_month in months:
        todo_url = f"http://tianqi.2345.com/t/wea_history/js/{selected_month}/58238_{selected_month}.js"
        """ 
            这里默认读出来的数据是乱码，实例如下：
            2021-11-01',bWendu:'23Ўж',yWendu:'14Ўж',tianqi:'¶аФЖ',fengxiang:'¶«ұұ·з',fengli:'2ј¶',aqi:'74',aqiInfo:'Бј',aqiLevel:'2'},{
        """     
        data = requests.get(todo_url,headers=headers)

        data = data.text.lstrip("var weather_str=").rstrip(';')
        month_list = data.split('ymd:\'')[1:]
        for line in month_list:
            date = line[:10]
            if 'Ўж' in data:
                temp_high = int(line[10:].split('Ўж')[0].split(':\'')[1])
                temp_low = int(line[10:].split('Ўж')[1].split(':\'')[1])
            elif 'Ёц' in data:
                temp_high = int(line[10:].split('Ёц')[0].split(':\'')[1])
                temp_low = int(line[10:].split('Ёц')[1].split(':\'')[1])  

            date_list.append(date)
            temp_high_list.append(temp_high)
            temp_low_list.append(temp_low)
    return date_list,temp_high_list,temp_low_list
        


In [12]:
date_list,temp_high_list,temp_low_list = get_history_temps(months=months)


In [13]:
date_list

['2020-01-01',
 '2020-01-02',
 '2020-01-03',
 '2020-01-04',
 '2020-01-05',
 '2020-01-06',
 '2020-01-07',
 '2020-01-08',
 '2020-01-09',
 '2020-01-10',
 '2020-01-11',
 '2020-01-12',
 '2020-01-13',
 '2020-01-14',
 '2020-01-15',
 '2020-01-16',
 '2020-01-17',
 '2020-01-18',
 '2020-01-19',
 '2020-01-20',
 '2020-01-21',
 '2020-01-22',
 '2020-01-23',
 '2020-01-24',
 '2020-01-25',
 '2020-01-26',
 '2020-01-27',
 '2020-01-28',
 '2020-01-29',
 '2020-01-30',
 '2020-01-31',
 '2020-02-01',
 '2020-02-02',
 '2020-02-03',
 '2020-02-04',
 '2020-02-05',
 '2020-02-06',
 '2020-02-07',
 '2020-02-08',
 '2020-02-09',
 '2020-02-10',
 '2020-02-11',
 '2020-02-12',
 '2020-02-13',
 '2020-02-14',
 '2020-02-15',
 '2020-02-16',
 '2020-02-17',
 '2020-02-18',
 '2020-02-19',
 '2020-02-20',
 '2020-02-21',
 '2020-02-22',
 '2020-02-23',
 '2020-02-24',
 '2020-02-25',
 '2020-02-26',
 '2020-02-27',
 '2020-02-28',
 '2020-02-29',
 '2020-03-01',
 '2020-03-02',
 '2020-03-03',
 '2020-03-04',
 '2020-03-05',
 '2020-03-06',
 '2020-03-

In [14]:
import pandas


temp_data = {
    'date': date_list,
    'max_temp': temp_high_list,
    'min_temp': temp_low_list
    }
df = pandas.DataFrame(temp_data)



In [15]:
df

,date,max_temp,min_temp
0,2020-01-01,7,5
1,2020-01-02,9,6
2,2020-01-03,7,6
3,2020-01-04,9,7
4,2020-01-05,11,11
...,...,...,...
1030,2022-10-27,19,13
1031,2022-10-28,19,13
1032,2022-10-29,20,11
1033,2022-10-30,20,14


In [16]:
df['date'] = df['date'].apply(lambda x: x.replace('-',''))
df['date'] = pandas.to_datetime(df['date'])

In [18]:
df.to_csv(
    os.path.join(
        DataDir.Raw,
        "History_temperature_{}_{}.csv".format(
        months[0],months[-1]
    ))    ,
    encoding = 'utf-8-sig'

)